In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import subprocess
import os

from pdflatex import PDFLaTeX

In [2]:
students = pd.read_excel('1 курс БПМ.xlsx')
students.head()

,Фамилия,Имя,Отчество,Курс,Группа
0,Абдыев,Араз,Акшин оглы,1 курс,БПМ202
1,Алеканкин,Кирилл,Олегович,1 курс,БПМ205
2,Аманов,Айнар,Нурланович,1 курс,БПМ204
3,Амиров,Эмир,Шаиг оглы,1 курс,БПМ201
4,Андриянов,Павел,Павлович,1 курс,БПМ203


In [3]:
# в этом году имя и фамилия студента уникальный идентификатор

set_students_name = [set([x, y]) for x, y in zip(students['Фамилия'].to_list(),students['Имя'].to_list())]
set_students_full_name = [set([x, y, z]) for x, y,z in zip(students['Фамилия'].to_list(),students['Имя'].to_list(), 
                                                        students['Отчество'].to_list())]
set_students_name[:3]

[{'Абдыев', 'Араз'}, {'Алеканкин', 'Кирилл'}, {'Айнар', 'Аманов'}]

In [8]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('NIS2021-2-6b794392fe59.json', scope)
client = gspread.authorize(creds)

In [9]:
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open('2021Оценочный лист проекта (Ответы)').sheet1

# Extract and print all of the values
list_of_hashes = sheet.get_all_records()
#print(list_of_hashes[0])

In [11]:
table = pd.DataFrame(list_of_hashes)
table = table.loc[table['сгенерирован оценочный лист']!=1]
table

,Отметка времени,Адрес электронной почты,Название проекта,ФИО руководителя проекта,Должность руководителя проекта,ФИО участника проекта (для групповых проектов оценочный лист заполняется на каждого участника группы),Группа участника проекта (необязательный вопрос),"Оценка результата/продукта проекта (Опр), 0.7 от результирующей оценки",Примечания к оценке Опр (при необходимости),"Оценка отчета по проекту (Оз), 0.3 от результирующей оценки",Примечания к оценке Оз (при необходимости),"Рекомендуете ли Вы эту работу для выступления, как одну из лучших?",status,сгенерирован оценочный лист,записано в таблицу нис,result,
99,05.07.2021 11:50:41,mguskova@miem.hse.ru,Первый проект,Иванов Иван Иванович,Преподаватель,Петров Иван Сергеевич,БПМ201,7,,8,,,,,,7,
100,05.07.2021 11:51:23,mguskova@miem.hse.ru,Второй проект,Иванов Иван Иванович,Преподаватель,Петрова Екатерина Сергеевна,БПМ202,8,,7,,,,,,8,


In [12]:
table.columns

Index(['Отметка времени', 'Адрес электронной почты', 'Название проекта',
       'ФИО руководителя проекта', 'Должность руководителя проекта',
       'ФИО участника проекта (для групповых проектов оценочный лист заполняется на каждого участника группы)',
       'Группа участника проекта (необязательный вопрос)',
       'Оценка результата/продукта проекта (Опр), 0.7 от результирующей оценки',
       'Примечания к  оценке Опр (при необходимости)',
       'Оценка отчета по  проекту (Оз), 0.3 от результирующей оценки',
       'Примечания к  оценке Оз (при необходимости)',
       'Рекомендуете ли Вы эту работу для выступления, как одну из лучших? ',
       'status', 'сгенерирован оценочный лист', 'записано в таблицу нис',
       'result', ''],
      dtype='object')

In [13]:
ren = {'Адрес электронной почты':'email',
       'Группа участника проекта (необязательный вопрос)':'group',
       'Должность руководителя проекта':'position',
       'Название проекта':'name',
       'Отметка времени':'date',
       'Оценка отчета по  проекту (Оз), 0.3 от результирующей оценки':'Oz',
       'Оценка результата/продукта проекта (Опр), 0.7 от результирующей оценки':'Opr',
       'Результирующая оценка':'results',
       'ФИО руководителя проекта':'supervisor',
       'ФИО участника проекта (для групповых проектов оценочный лист заполняется на каждого участника группы)':'student',
       'Примечания к  оценке Оз (при необходимости)':'prim2',
       'Примечания к  оценке Опр (при необходимости)':'prim1'
    
}

In [14]:
df = table.copy()
df.rename(columns=ren,inplace=True)
df

,date,email,name,supervisor,position,student,group,Opr,prim1,Oz,prim2,"Рекомендуете ли Вы эту работу для выступления, как одну из лучших?",status,сгенерирован оценочный лист,записано в таблицу нис,result,
99,05.07.2021 11:50:41,mguskova@miem.hse.ru,Первый проект,Иванов Иван Иванович,Преподаватель,Петров Иван Сергеевич,БПМ201,7,,8,,,,,,7,
100,05.07.2021 11:51:23,mguskova@miem.hse.ru,Второй проект,Иванов Иван Иванович,Преподаватель,Петрова Екатерина Сергеевна,БПМ202,8,,7,,,,,,8,


In [15]:
def run_util(cmd, folder, prnt=True):
    '''
        Run (console) program (utility) and return it output.
    '''
    if type(cmd) == str:
        cmd = cmd.split()
    if prnt:
        print('Running:' , ' '.join(cmd))
    output = subprocess.Popen(cmd, stdout=subprocess.PIPE, cwd=folder).communicate()[0]#.decode("utf-8")
    return output

In [16]:
def find_group(student):
    st = set(student.split())
    assert st in set_students_name or st in set_students_full_name, st
    for i in range(len(set_students_name)):
        if st == set_students_full_name[i]:
            return students.iloc[i]['Группа']
    

In [17]:
def find_full_name(student):
    st = set(student.split())
    #assert st in set_students_name or st in set_students_full_name, st
    if not (st in set_students_name or st in set_students_full_name):
        print(st)
    
    if st in set_students_full_name:
        return student
    for i in range(len(set_students_name)):
        if st == set_students_name[i]:
            return students.iloc[i]['Фамилия']+' '+students.iloc[i]['Имя']+' '+students.iloc[i]['Отчество']
    return student
     

In [79]:
df.student = df.student.apply(find_full_name) 
df.loc[df.group==''].group = df.loc[df.group==''].student.apply(find_group)

In [18]:
def replace(st, i):
    st = st.replace('NAME', df.name.iloc[i])
    st = st.replace('SUPERVISOR', df.supervisor.iloc[i])
    st = st.replace('POSITION', df.position.iloc[i])
    st = st.replace('STUDENT', df.student.iloc[i])
       
    st = st.replace('GROUP', df.group.iloc[i])
    st = st.replace('MARK1', str(df.Opr.iloc[i]))
    st = st.replace('MARK2', str(df.Oz.iloc[i]))
    
    st = st.replace('RESULT', str(df.result.iloc[i]))
    st = st.replace('PRIM1', str(df.prim1.iloc[i]))
    st = st.replace('PRIM2', str(df.prim2.iloc[i]))
    st = st.replace('DATE', str(df.date.iloc[i].split(' ')[0]))
    kze = 0 if df.result.iloc[i] < 4 else 2
    st = st.replace('RESKZE', str(kze))
    return st

In [19]:
folder = 'оценочные листы/'
for i in range(df.shape[0]):
    with open('mark_template_py.tex', 'r', encoding='utf8') as template:
        file_name = 'оценочный лист ' + df.supervisor.iloc[i]+\
                  ' '+df.student.iloc[i]+' '+df.group.iloc[i] + '.tex'
        if os.path.exists(folder + file_name):
            os.remove(folder + file_name)
            os.remove(folder + file_name[:-4] + '.pdf')
            
        with open(folder + file_name, 'a', encoding='utf8') as file:
            lst = template.readlines()
            for st in lst:
                file.write(replace(st, i))
        command = f'pdflatex.exe -synctex=1 -interaction=nonstopmode "%s".tex'%file_name[:-4]
        run_util(command, folder);



Running: pdflatex.exe -synctex=1 -interaction=nonstopmode "оценочный лист Иванов Иван Иванович Петров Иван Сергеевич БПМ201".tex
Running: pdflatex.exe -synctex=1 -interaction=nonstopmode "оценочный лист Иванов Иван Иванович Петрова Екатерина Сергеевна БПМ202".tex


In [69]:
df[['name', 'supervisor', 'student', 'group', 'Opr', 'Oz','result']].to_csv('current.csv', index=False, sep=';')